<h1>Туториал по теме "Синтаксические парсеры русского языка"</h1>

Цель данной работы заключается в создании парсера русского языка, т.е. программы, которая на вход бы принимала произвольный текст, а на выходе выдавала бы его синтаксическую структуру. Например, так:

> "Мама мыла раму":

> (предложение
    (именная гр. (сущ мама))
    (глаг. гр. (глаг мыла)
        (именная гр. (сущ раму)))
    (. .)))

Это называется синтаксическим деревом предложения. В графическом виде его можно представить следующим образом (в упрощенном виде):

![Tree of sentence](https://habrastorage.org/storage2/951/7d8/1f4/9517d81f4df9c48bbb35864f98e70f3a.png)

Существует два основных подхода при создании синтаксического парсера:

* Метод, основанный на правилах (rule based)
* Машинное обучение с учителем (supervised machine learning)

Метод, основанный на правилах, применяется практически во всех коммерческих системах, т.к. он дает наибольшую точность. Основная идея заключается в создании набора правил, которые определяют, как проставлять связи в предложении. Например, в нашем предложении «Мама мыла раму» мы можем применять следующие правила (допустим, что мы уже сняли все неоднозначности и точно знаем грамматические категории слов):

слово («мама») в именительном падеже, женском роде и единственном числе должно зависеть от глагола («мыла») в единственном числе, прошедшем времени, женском роде и тип связи должен быть «субъект»
слово («раму») в винительном падеже должно зависеть от глагола и тип связи должен быть «объект».

Данный подход очень ресурсоемок, т.к. для создания парсера требуется хорошая команда лингвистов, которые должны буквально описать весь русский язык. Поэтому нам более интересен второй подход — машинное обучение с учителем.

Идея в парсинге, использующем машинном обучение, как и во всех остальных задачах машинного обучения довольно таки проста: мы даем компьютеру много примеров с правильными ответами, на которых система должна обучиться самостоятельно. Чтобы обучить синтаксические парсеры — в качестве данных для обучения используют специально размеченные корпуса (treebanks), коллекции текстов, в которых размечена синтаксическая структура. Наше предложение в таком корпусе может выглядеть следующим образом:

> 1	Мама	сущ.им.ед.жен.	2	субъект

> 2	мыла	глаг.ед.жен.прош	0	-

> 3	раму	сущ.вин.ед.жен.	2	объект

В этом формате мы записываем каждое предложение в виде строки, где каждая строка описывает отдельное слово в виде записей разделенных табуляцией. Для каждого слова нам нужно хранить следующие данные: 

* номер слова в предложении (1)
* словоформа (мама)
* грамматические категории (сущ.им.ед.жен.)
* номер главного слова (2)
* тип связи (субъект)

<h2>Размеченный корпус</h2>

Чтобы обучить новую языковую модель нам потребуется размеченный корпус. Для русского языка был выбран корпус от команды <a href="http://opencorpora.org"><b>OpenCorpora</b></a>. Корпус представляет собой набор текстов размеченных в формате XML:

Для начала нам нужно распарсить XML документ и определить необходимые данные.

In [6]:
def read_data(corpus_root):
    conllu_set = []
    for root, dirs, files in os.walk(corpus_root):
        for filename in files:
            if filename.endswith(".xml"):
                xmldoc = parse(root + '/' + filename)
                sentencelist = xmldoc.getElementsByTagName('sentence')
                for sentence in sentencelist:
                    for token in sentence.getElementsByTagName('token'):
                        text = r''.join(token.attributes['text'].value).encode('utf-8')
                        id_token = r''.join(token.getElementsByTagName('l')[0].attributes['id'].value).encode('utf-8')
                        inf_token = [ r''.join(g.attributes['v'].value).encode('utf-8') for g in token.getElementsByTagName('g')]
                        conllu_set.append(to_conllu(text, id_token, inf_token))
    return conllu_set

Собственно сам метод  для перевода даннных в формат malttab.

In [2]:
def to_conllu(text, id_token, inf_token):
    return str(text) + ' ' + '.'.join(inf_token[1:]) + ' ' + str(id_token) + ' ' + str(inf_token[0])

Приступим к практике. Импортируем необходмые классы:

In [1]:
from nltk.parse.malt import MaltParser
import os
import string
import collections
from xml.dom.minidom import parse
from nltk import word_tokenize, pos_tag

Наша программа будет состоять из следующих частей:
* Считываем набор
* Определяем тестовые и тренировачные данные.
* Обучаем систему.
* Токенезируем (делим на слова) наш "рабочий" текст.
* При помощи метода pos_tag определяем части речи.
* При помощи метода parse класса MaltParser выполняем синтаксический анализ
* Результат выводим на экран.

**Все методы задействованные в процессе будут рассмотрены ниже**

In [ ]:
#указываем директорию с нашим размеченным корпусом
corpus_root = 'corpus'
#считываем данные
reader = read_data(corpus_root)
data = list(reader)
#тренировочные данные
training_samples = data[:int(len(data) * 0.9)] # 3546 sentences or >1000000 words  
#тестовые данные
test_samples = data[int(len(data) * 0.9):] # 154 sentences or >100000 words

Есть несколько открытых парсеров, которые можно обучить для работы с русским языком. Вот два, которые наиболее популярные:

* MST Parser, основанных на задаче нахождения минимального остовного дерева
* MaltParser, основан на машинном обучении (хотя и MST Parser тоже, но там немного другая идея)

Обучение MST-парсера занимает гораздо больше времени и он также дает худшие результаты по сравнению с MaltParser, поэтому далее мы сфокусируемся на втором из них. Еще одним главным преимуществом данного парсера, он входит в состав пакета NLTK. 

In [2]:
#инициализация инстанса MaltParser
parser_dirname = 'maltparser-1.9.2'
mp = MaltParser(parser_dirname)

Обучим нашу модель.

In [4]:
#обучаем модель
mp.train_from_file(os.getcwd() + '\corpus\\ru_syntagrus-ud-train.conllu', verbose=False)

Получим следующее синтаксическое дерево.

In [7]:
#парсим наше предложение
mp.sentence = r'Помощь этой стране обычно поступает извне.'
sent1 = word_tokenize(mp.sentence)
print(mp.parse_one(sent1, top_relation_label='root'))

1	Помощь	_	NN	NN	_	0	a	_	_

2	этой	_	NN	NN	_	0	a	_	_

3	стране	_	NN	NN	_	0	a	_	_

4	обычно	_	NN	NN	_	0	a	_	_

5	поступает	_	NN	NN	_	0	a	_	_

6	извне	_	NN	NN	_	0	a	_	_

7	.	_	.	.	_	0	a	_	_




defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x0000020390A80BF8>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [5]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'NOUN',
                 'deps': defaultdict(<class 'list'>, {'nmod': [3]}),
                 'feats': 'Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing',
                 'head': 5,
                 'lemma': 'помощь',
                 'rel': 'nsubj',
                 'tag': '_',
                 'word': 'Помощь'},
             2: {'address': 2,

Синтаксическое дерево на основе уже обученной модели для английского языка.

In [ ]:
mp = MaltParser("maltparser-1.9.2","engmalt.poly-1.7.mco")
mp.sentence = ''
sent1 = word_tokenize('What is your name')
print(mp.parse_one(sent1).tree())